# 4-3. Gradio로 Agent 챗봇 UI 만들기

> 📕이 실습은 `11_Agent.ipynb` 노트북과 동일한 Colab 세션에서 이어서 실행하는 것을 전제로 합니다. Gradio UI가 작동하려면, 이전 실습에서 생성된 agent_with_memory 객체와 config 변수가 현재 세션의 메모리에 로드되어 있어야 합니다. `agent_with_memory`와 `config` 변수를 메모리에 준비시킨 후, 이 `4-3` 실습을 진행해야 합니다.

In [1]:
# LangChain 1.0+ 설치 (핵심, 커뮤니티, 파트너 패키지 포함)
# langchain: Agent, LCEL(LangChain Expression Language) 등 핵심 기능
!pip install -U langchain
# langchain-core: LangChain의 기본 데이터 구조 (Schema, Runnable 등)
!pip install -U langchain-core
# langchain-community: ChromaDB, TextLoader 등 커뮤니티 기반 연동 모듈
!pip install -U langchain-community
# langchain-upstage: Upstage LLM, Embedding 모델 연동 패키지
# (구버전과 호환성 문제가 있을 수 있어 -U로 업그레이드 설치 권장)
!pip install -U langchain-upstage

# RAG 파이프라인 관련
# langchain-text-splitters: 문서를 청크로 분할하는 도구
!pip install langchain-text-splitters
# tiktoken: 텍스트를 토큰 단위로 계산하는 라이브러리
!pip install tiktoken

# Vector Store
# chromadb: 로컬(인메모리 또는 파일) 기반의 벡터 데이터베이스
!pip install chromadb

# Document Loaders (PDF 처리용)
!pip install pypdf pymupdf pypdfium2

# Agent의 대화 기록을 SQLite DB에 저장하기 위한 라이브러리
!pip install langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
from google.colab import drive
from dotenv import load_dotenv
from pprint import pprint
from os import getenv
import sqlite3
import glob
# langchain-community에서 로더(PDF, TXT) 임포트
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
# data chunking을 위한 텍스트 스플리터
from langchain_text_splitters import RecursiveCharacterTextSplitter
# langchain-community에서 Vector Store (Chroma) 임포트
from langchain_community.vectorstores import Chroma
# Upstage 임베딩 모델 임포트
from langchain_upstage import UpstageEmbeddings
from langchain_classic.tools.retriever import create_retriever_tool
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_upstage import ChatUpstage
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.runnables import RunnableConfig

# 1. Google Drive 마운트 (드라이브에 저장된 .env 파일을 접근하기 위함)
drive.mount('/content/drive')

# 2. .env 파일이 저장된 기본 경로를 설정합니다.
# (본인의 구글 드라이브 환경에 맞게 이 경로를 수정해야 합니다.)
base_path = '/content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/'



Mounted at /content/drive


최초 실행시

In [3]:
# base_path에 지정된 디렉토리가 존재하지 않을 경우 생성 (-p 옵션).
# .env 파일을 저장할 폴더를 미리 준비함.
!mkdir -p '{base_path}'

# Colab 환경에서 .env 파일을 생성하고 API 키를 저장하는 명령어.
# {your_api_key} 부분에 본인의 실제 Upstage API 키를 입력해야 함.
# '!'는 Colab에서 셸 명령어를 실행함을 의미.
!echo 'UPSTAGE_API_KEY={your_api_key}' > '{base_path}.env'


In [4]:

# 3. .env 파일 로드
# base_path 경로에 있는 .env 파일을 찾아 환경 변수로 로드합니다.
# load_dotenv(".env") # 로컬 환경용
load_dotenv(base_path + ".env")

# 4. 환경 변수 읽기
# 런타임 환경에 로드된 'UPSTAGE_API_KEY' 값을 읽어옵니다.
UPSTAGE_API_KEY = getenv("UPSTAGE_API_KEY")

# API 키가 성공적으로 로드되었는지 확인하고 메시지를 출력.
if UPSTAGE_API_KEY:
    print('Success API Key Setting!')
else:
    print(f'ERROR: Failed to load UPSTAGE_API_KEY from {base_path}')

Success API Key Setting!


In [5]:
# 1. DB 및 테이블 생성 함수 정의
def setup_refined_database():
    # base_path에 'orders.db'라는 파일 기반 DB를 생성(연결)
    conn = sqlite3.connect(base_path + 'orders.db')
    # 커서(Cursor)는 DB에 SQL 명령어를 실행하는 객체
    c = conn.cursor()
    try:
        # 'orders'라는 이름의 테이블(표) 생성
        # (order_id, customer_name, item_name, status 4개의 열을 가짐)
        c.execute('''
            CREATE TABLE orders (
                order_id TEXT PRIMARY KEY,
                customer_name TEXT,
                item_name TEXT,
                status TEXT
            )
        ''')
        # 샘플 데이터(가상 주문 내역) 삽입
        # Delivered         : 배송 완료
        # Shipping Delayed  : 배송 지연
        # Processing        : 처리 중
        c.executemany('INSERT INTO orders (order_id, customer_name, item_name, status) VALUES (?, ?, ?, ?)', [
            ('ORDER123', 'Alice Smith', 'Introduction to Python', 'Delivered'),
            ('ORDER456', 'Bob Johnson', 'The AI Revolution', 'Shipping Delayed'),
            ('ORDER789', 'Charlie Lee', 'Data Structures', 'Processing'),
            ('ORDER101', 'David Kim', 'Learning SQL', 'Processing'),
            ('ORDER102', 'Alice Smith', 'Advanced ML', 'Delivered'),
            ('ORDER505', 'Eva Moon', 'Cloud Computing', 'Shipping Delayed')
        ])
        # 변경사항을 DB 파일에 최종 저장(Commit)
        conn.commit()
        print("--- Mock 'orders.db' 생성 완료 ---")
    except sqlite3.OperationalError:
        # 테이블이 이미 존재하면(두 번째 실행 시) 이 메시지를 출력
        print("--- Mock 'orders.db'가 이미 존재합니다 ---")
    finally:
        # 작업이 끝나면 DB 연결 종료
        conn.close()

# DB 셋업 함수 실행
setup_refined_database()

--- Mock 'orders.db' 생성 완료 ---


In [6]:
# 1. Load: data 폴더의 모든 PDF와 shipping_policy.txt 파일을 로드
!unzip -o "{base_path}/data.zip" -d "{base_path}"
pdf_files = glob.glob(base_path + '/data/*.pdf') # PDF 파일 경로
documents = []
for pdf_filepath in pdf_files:
    loader = PyMuPDFLoader(pdf_filepath) # PyMuPDF로 PDF 로드
    documents.extend(loader.load())

# 배송 정책 TXT 파일 로드
shipping_loader = TextLoader(base_path + "shipping_policy.txt")
documents.extend(shipping_loader.load())
print(f"총 {len(documents)}개의 문서를 로드했습니다.")

# 2. Chunk: 로드된 문서를 200자 단위(overlap 20자)로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # 청크 크기
    chunk_overlap=20, # 중복 크기
)
chunks = text_splitter.split_documents(documents)
print(f"문서를 총 {len(chunks)}개의 청크로 분할했습니다.")

# 3. Embed & Store: Upstage 임베딩 모델로 청크를 벡터화하여 ChromaDB에 저장
embeddings = UpstageEmbeddings(model="embedding-query") # Upstage 임베딩 모델

# ChromaDB에 청크들을 임베딩하여 저장
# persist_directory: 벡터 데이터를 디스크에 저장할 경로
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=base_path + "chroma_db_v1"
)
print(f"Vector Store 생성 완료. 컬렉션 개수: {vectorstore._collection.count()}")

# 4. Retriever: 저장된 Vector Store를 '검색기(Retriever)'로 변환
retriever = vectorstore.as_retriever()
print("RAG Retriever가 준비되었습니다.")

Archive:  /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio//data.zip
   creating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/도서 품절 보상제도 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/매장 픽업 서비스_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/무료배송 추가적립_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/배송지연 보상제도_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/신규 회원 혜택_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/영원한 YES포인트_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/제휴 할인카드_서비스 혜택 - 예스24.pdf  
  inflating: /content/drive/MyDrive/Colab Notebooks/AI/12_Gradio/data/총알배송_서비스 혜택 - 예스24.pdf  
총 9개의 문서를 로드했습니다.
문서를 총 81개의 청크로 분할했습니다.
Vector Store 생성 완료. 컬렉션 개수: 8

In [7]:
# create_retriever_tool: Retriever 객체를 Agent용 Tool로 변환하는 헬퍼 함수
retriever_tool = create_retriever_tool(
    retriever, # 1. 사용할 검색기(Retriever)
    "Policy_Search", # 2. Agent가 부를 도구 이름 (공백X, 영어 권장)
    # 3. Agent가 이 도구의 용도를 파악할 수 있는 상세한 설명 (가장 중요!)
    """배송 정책, 보상 정책, 환불 정책 등 AI 온라인 서점의
    다양한 정책 문서에서 고객 질문과 관련된 정보를 검색합니다.
    (예: "배송 지연 시 보상 정책이 어떻게 되나요?")
    """,
)

# @tool: 이 데코레이터가 붙은 함수를 LangChain Tool로 자동 변환
@tool
def get_order_status_from_db(order_id: str) -> str:
    """(DB 조회)
        주문 ID를 받아 현재 '배송 상태' 문자열을 반환합니다.
        단, DB 연결 오류 시 "DB Error: {오류 메시지}" 반환하고,
        주문 ID가 없을 시 "Order Not Found"를 반환합니다.
    """
    # 디버깅용 (Agent가 이 함수를 호출할 때마다 콘솔에 출력됨)
    print(f"--- get_order_status_from_db 호출됨: {order_id} ---")
    try:
        # 0-3에서 만든 DB 파일에 연결
        conn = sqlite3.connect(base_path + 'orders.db')
        c = conn.cursor()
        # SQL 쿼리 실행: orders 테이블에서 order_id가 일치하는 row의 'status' 열을 선택
        c.execute("SELECT status FROM orders WHERE order_id = ?", (order_id,))
        # 결과 중 첫 번째 줄을 가져옴 (예: ('Delivered',))
        result = c.fetchone()
        conn.close()

        if result:
            return result[0] # 튜플이 아닌 문자열 값(예: 'Delivered')을 반환
        else:
            return "Order Not Found"
    except Exception as e:
        return f"DB Error: {str(e)}"

# 3. 배송 지연 쿠폰 발급 도구 함수
@tool
def issue_complaint_coupon(order_id: str) -> str:
    """(DB 조회 기반 쿠폰 발급)
        주문 ID를 받아, 배송 상태가 'Shipping Delayed'일 때만 쿠폰을 발급합니다.
        (실제 쿠폰 생성 로직은 하드코딩으로 대체하고, 발급 메시지만 반환합니다.)
    """
    # 디버깅용
    print(f"--- issue_complaint_coupon 호출됨: {order_id} ---")

    # (중요!) 도구가 다른 도구를 호출할 때는 .invoke() 사용
    order_status = get_order_status_from_db.invoke(order_id)

    # 비즈니스 로직: 배송 지연 상태가 맞는지 확인
    if order_status == "Shipping Delayed":
        # (원래라면 이곳에 쿠폰 DB INSERT 로직 등이 들어감)
        return f"주문 ID {order_id}에 대해 5,000원 할인 쿠폰이 발급되었습니다."
    else:
        return f"주문 ID {order_id}는 쿠폰 발급 대상이 아닙니다. 현재 상태: {order_status}"

system_prompt = """
당신은 AI 온라인 서점의 고객 서비스 AI 에이전트입니다.
당신은 사용자의 요청을 해결하기 위해 '도구(tool)'를 사용해야 합니다.

[작업 수행 가이드라인]
1.  사용자의 요청을 받으면, 먼저 '생각(Thought)'을 합니다.
2.  요청을 해결하는 데 필요한 정보가 무엇인지 분석합니다.
3.  필요한 정보를 수집하기 위해 어떤 '도구'를 사용해야 할지 결정합니다.
4.  도구를 사용한 후, '관찰(Observation)' 결과를 봅니다.
5.  모든 정보가 모일 때까지 1-4 단계를 반복합니다.
6.  모든 정보가 수집되면, 이를 바탕으로 사용자에게 최종 답변을 생성합니다.

[!!절대적 규칙!!]
- 'issue_complaint_coupon' 도구는 고객이 요청하더라도, 'get_order_status_from_db' 도구의 결과가 'Shipping Delayed'일 때만 사용해야 합니다.
- 주문 상태가 불명확하면, 쿠폰을 발급하기 전에 반드시 'get_order_status_from_db'를 먼저 사용해야 합니다.
- 절대 추측하지 말고, 항상 도구의 실행 결과를 바탕으로만 답변하세요.
- 정책에 대한 질문은 'Policy_Search' 도구를 사용하세요.
"""

In [8]:
# 1. Agent가 사용할 도구들을 리스트로 묶기
agent_tools = [retriever_tool, get_order_status_from_db, issue_complaint_coupon]

# 2. Agent의 '두뇌'가 될 LLM 정의
llm = ChatUpstage()

# 1. 대화 기록을 저장할 DB(database.db)와 연결하는 Checkpointer 생성
# (check_same_thread=False는 sqlite가 여러 스레드에서 접근 가능하도록 허용하는 설정)
checkpointer = SqliteSaver(
    sqlite3.connect(base_path + "database.db", check_same_thread=False)
)

# 2. (선택) Agent가 'messages' 외에 추가로 기억할 상태 정의
# class CustomAgentState(AgentState):
#     user_id: str # 예: 사용자 ID
#     preferences: dict # 예: 사용자 선호도

# 3. Agent를 다시 생성하되, 'checkpointer' 옵션을 추가
agent_with_memory = create_agent(
    model=llm,
    tools=agent_tools,
    system_prompt=system_prompt,
    # state_schema=CustomAgentState, # (만약 CustomAgentState를 쓴다면)
    checkpointer=checkpointer # (중요!) Agent에 메모리 시스템 장착
)
print("메모리가 적용된 Agent 생성 완료")

# '2번 고객'과의 대화방을 설정 (이 ID를 바꾸면 다른 대화방이 됨)
config: RunnableConfig = {"configurable": {"thread_id": "2"}}

메모리가 적용된 Agent 생성 완료


------------------

이전 `4-2` 실습에서는 Colab의 코드 셀에서 `.invoke()`를 실행하여 Agent와 대화했습니다. 이번 세션에서는 **Gradio** 라이브러리를 사용해, 우리가 만든 `agent_with_memory`에 실제 챗봇과 같은 **웹 UI(Web UI)** 를 입혀보는 작업을 진행합니다.


**학습 목표**
1. Gradio가 무엇인지, 왜 사용하는지 이해합니다.

2. `gr.ChatInterface`를 사용해 Python 함수를 웹 UI로 감싸는 방법을 배웁니다.

3. `4-2`에서 만든 Agent를 Gradio와 연결하여 실제 챗봇 데모를 완성합니다.

4. Gradio의 현실적인 한계와 사용처를 파악합니다.

## 1. Gradio란 무엇인가?
**Gradio**는 머신러닝 모델이나 데이터 과학 스크립트를 위한 **웹 UI를 구축하는 가장 빠른 방법**을 제공하는 Python 라이브러리입니다.

몇 줄의 코드만으로 Python 함수를 감싸서, 텍스트 박스, 버튼, 이미지 업로드 등이 포함된 인터랙티브한 웹 데모를 만들 수 있습니다.

> 우리가 `4-2`에서 만든 `agent_with_memory`가 '챗봇 엔진'이라면, **Gradio**는 이 엔진을 장착하여 사용자가 직접 운전할 수 있도록 '자동차의 운전석과 대시보드'를 만들어주는 도구입니다.

## 2. 실습 준비
### 2-1. Gradio 설치

In [9]:
# Gradio 라이브러리 설치
!pip install gradio

## 3. Gradio 인터페이스 구축
### 3-1. Agent를 위한 래퍼(Wrapper) 함수 정의

Gradio는 `fn(input_1, input_2, ...)` 형태의 간단한 Python 함수를 필요로 합니다. 하지만 우리가 만든 `agent_with_memory`는 `invoke({"messages": [...]}, config=...)`와 같이 복잡한 딕셔너리 입력을 받습니다.

따라서 Gradio와 Agent 사이를 연결해 줄 '번역기' 역할의 **래퍼(Wrapper) 함수**를 정의합니다.

Gradio의 여러 컴포넌트 중, 챗봇에 가장 특화된 `gr.ChatInterface`를 사용하겠습니다. `gr.ChatInterface`는 `fn(message, history)`라는 두 개의 인자를 받는 함수를 필요로 합니다.

- `message`: 사용자가 방금 입력한 **새 메시지 (문자열)**

- `history`: 이전 대화 기록 (Gradio가 자동으로 관리해 줌)

In [10]:
print('Gradio 래퍼 함수를 정의합니다...')

def agent_chat_wrapper(message, history):
    """
    Gradio의 ChatInterface가 호출할 래퍼 함수입니다.

    Args:
        message (str): 사용자가 새로 입력한 텍스트.
        history (list): Gradio가 관리하는 대화 기록 (이번 실습에서는 사용 X).
    """
    print(f"[Gradio 입력] Message: {message}")
    print(f"[Gradio 입력] History: {history}")

    # 4-2에서 만든 checkpointer가 DB에서 대화 기록을 관리하므로,
    # 우리는 Agent에게 '새 메시지'만 전달하면 됩니다.
    # config 변수는 4-2에서 정의한 것을 그대로 사용합니다. (예: thread_id="1")
    input_data = {"messages": [{"role": "user", "content": message}]}

    # 4-2에서 만든 Agent를 호출합니다.
    result = agent_with_memory.invoke(input_data, config)

    # Agent의 응답(AIMessage)에서 텍스트 내용만 추출합니다.
    response = result['messages'][-1].content

    print(f"[Agent 응답] Response: {response}")

    # Gradio에 최종 텍스트 응답을 반환합니다.
    return response

Gradio 래퍼 함수를 정의합니다...


### 3-2. Gradio 인터페이스 실행
이제 래퍼 함수를 `gr.ChatInterface`에 연결하고 `.launch()`를 호출하여 웹 UI를 실행합니다.

In [11]:
import gradio as gr

print('Gradio ChatInterface를 실행합니다...')

# gr.ChatInterface: 챗봇 UI를 자동으로 생성합니다.
# - fn: 사용자가 메시지를 보낼 때마다 호출될 함수 (우리가 만든 래퍼 함수)
# - title: 챗봇 UI 상단에 표시될 제목
demo = gr.ChatInterface(
    fn=agent_chat_wrapper,
    title="🤖 AI 온라인 서점 고객센터",
    theme="soft" # UI 테마 설정 (선택 사항)
)

# .launch(): Colab에서 Gradio 앱을 실행합니다.
# - share=True: 외부에서 접근 가능한 공개 URL을 생성합니다. (72시간 유효)
# - debug=True: 오류 발생 시 Colab 셀에 상세 로그를 출력합니다.
demo.launch(share=True, debug=True)

Gradio ChatInterface를 실행합니다...


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f34bffc4cbe3477e04.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Gradio 입력] Message: 배송 조회 하려면 뭐가 필요해?
[Gradio 입력] History: []
--- get_order_status_from_db 호출됨: 123456 ---
[Agent 응답] Response: 주문 ID가 '123456'인 주문은 데이터베이스에 존재하지 않습니다. 다른 주문 ID가 있다면 알려주세요.
[Gradio 입력] Message: 주문 ID ORDER456 이래
[Gradio 입력] History: [['배송 조회 하려면 뭐가 필요해?', "주문 ID가 '123456'인 주문은 데이터베이스에 존재하지 않습니다. 다른 주문 ID가 있다면 알려주세요."]]
--- get_order_status_from_db 호출됨: ORDER456 ---
[Agent 응답] Response: 주문 ID 'ORDER456'의 배송 상태는 '배송 지연'입니다. 이 주문에 대해 어떤 도움이 필요하신가요?
[Gradio 입력] Message: 배송 지연에 대한 보상을 해줄 수 있어?
[Gradio 입력] History: [['배송 조회 하려면 뭐가 필요해?', "주문 ID가 '123456'인 주문은 데이터베이스에 존재하지 않습니다. 다른 주문 ID가 있다면 알려주세요."], ['주문 ID ORDER456 이래', "주문 ID 'ORDER456'의 배송 상태는 '배송 지연'입니다. 이 주문에 대해 어떤 도움이 필요하신가요?"]]
--- issue_complaint_coupon 호출됨: ORDER456 ---
--- get_order_status_from_db 호출됨: ORDER456 ---
[Agent 응답] Response: 주문 ID 'ORDER456'에 대해 5,000원 할인 쿠폰이 발급되었습니다. 이 쿠폰을 사용하시려면 고객 서비스 팀에 문의해주세요.
[Gradio 입력] Message: 뭐야 쿠폰 왜 주는거야?
[Gradio 입력] History: [['배송 조회 하려면 뭐가 필요해?', "주문 ID가 '123456'인 주문은 데이터베

### 3-3. 결과 확인
위 셀을 실행하면 Colab 셀 내부에 챗봇 UI가 나타나고, `Running on public URL: ...`이라는 공개 주소가 출력됩니다.

이 주소를 클릭하여 새 탭에서 챗봇을 테스트해 보세요.

`4-2`에서 테스트했던 시나리오를 그대로 수행할 수 있습니다.

1. "배송 조회 하려면 뭐가 필요해?"

2. "주문 ID ORDER456 이래"

3. "뭐야 쿠폰 왜 주는거야?"

Agent가 이전 대화를 기억하고 `thread_id="2"`의 맥락에 맞춰 올바르게 답변하는 것을 웹 UI로 직접 확인할 수 있습니다.

## 4. Gradio의 역할과 현실적 한계

Gradio는 방금 경험했듯이 정말 빠르고 강력하지만, 실제 상용 서비스에서 메인으로 사용되지는 않습니다. 그 이유와 Gradio의 정확한 용도를 아는 것이 중요합니다.

**Gradio의 핵심 장점**
- **신속한 프로토타이핑:** ML 모델이나 Agent의 데모를 가장 빠르게 만들 수 있습니다.

- **HuggingFace 연동:** HuggingFace Spaces에 배포하여 전 세계에 모델을 공유하기 매우 쉽습니다.

- **내부 공유:** 코드를 모르는 기획자나 동료에게 "이렇게 작동합니다"라고 보여주는 '내부용 툴'로 완벽합니다.


**Gradio가 잘 쓰이지 않는 이유 (한계)**

- **제한된 UI 커스터마이징:** Gradio는 정해진 컴포넌트(버튼, 텍스트 박스 등)를 제공합니다. 우리가 원하는 대로 디자인(CSS)을 세밀하게 변경하거나 복잡한 UI/UX(예: 드래그 앤 드롭, 복잡한 레이아웃)를 구현하는 데 한계가 명확합니다.

- **복잡한 상태 관리의 어려움:** `gr.State`로 상태를 관리할 수 있지만, 수천 명의 사용자가 동시 접속하는 상용 서비스의 복잡한 세션 관리, 유저 인증 등을 처리하기에는 부족합니다.

- **확장성 및 성능:** Gradio 자체는 대규모 트래픽을 처리하는 프로덕션급 백엔드 서버로 설계되지 않았습니다.


**결론:** Gradio는 "상용 제품(Production)"을 만드는 도구가 아니라, "데모(Demo)"나 "내부용 PoC(Proof of Concept)"를 만드는 데 특화된 도구입니다.

실제 상용 서비스는 React, Vue 같은 프론트엔드 프레임워크로 UI를 만들고, FastAPI, Django 등으로 백엔드 API를 구축하여 Agent를 연동하는 방식을 사용합니다.